In [ ]:
import torch
import os
import gc
import json
import subprocess
import pandas as pd
from torch import cuda
from openai import OpenAI
from IPython.display import Markdown, display
import re
from google.colab import files, drive

drive.mount('/content/drive')

# USER INPUT
user_output_folder="Paste your Google Drive folder for storing LLMs output here"

user_base_url="https://openrouter.ai/api/v1"
user_api_key="Paste your OpenRouter API here"

user_model = "Paste LLMs OpenRouter address here"
user_filename_id = "Name the model here"
user_max_token = 16384

print("Done")

In [ ]:
class LLMSolver:
  def __init__(self, ulr=None, api=None, model=None, filename_id=None, token=0, folder=None):
    # Initialize OpenAI client with environment variable
    self.client=OpenAI(
        base_url=ulr,
        api_key=api,
    )
    self.model = model
    self.filename_id = filename_id
    self.max_token = token
    self.output_folder = folder

    # Setup Google Drive and output folder
    self.setup_drive()

  def setup_drive(self):
    """Mount Google Drive and create output folder"""
    try:
      drive.mount('/content/drive')
      output_folder = self.output_folder
      os.makedirs(output_folder, exist_ok=True)
      print(f"Markdown files will be saved to: {output_folder}")
    except Exception as e:
      print(f"Error setting up drive: {e}")

  def load_data(self):
    """Upload and load CSV file"""
    try:
      uploaded = files.upload()
      filename = list(uploaded.keys())[0]
      df = pd.read_csv(filename)
      display(df.head(20))
      return df
    except Exception as e:
      print(f"Error loading data: {e}")
      return None

  def prompt_design(self, mode):
      """Get the appropriate prompt template based on mode"""
      PROMPT_GSR = """
      Structure your responses using proper Markdown formatting with the following sections:
      Section 1: Problem summary
        Briefly restate the problem in your own words.
      Section 2: Given data
        List all the provided data with units and symbols using bullet points.
      Section 3: Find
        Clearly state what needs to be calculated.
      Section 4: Detailed Solution
        Show all calculation steps clearly and logically.
        Number each sub-step (e.g., **Step 4.1**, **Step 4.2**, etc.).
        Explain each step thoroughly, including assumptions, unit conversions, and equations used.
      Section 5: Solution Review
        Present the result with appropriate units and LaTeX formatting.
      Section 6: Final Answer

      **IMPORTANT FORMATTING REQUIREMENTS:**
        - Use proper Markdown syntax (##, -, **, etc.)
        - NEVER use square brackets [ ] and ( ) for math
        - Format all mathematical expressions, equations, and formulas using LaTeX:
          - Use $...$ for inline math.  E.g., $x = 5$
          - Use $$...$$ for display math. E.g., $$x = 5$$
        - Use **bold** for emphasis and step labels
        - Use bullet points (-) for lists except for math
        - Do NOT use HTML tags like <h2>, <p>, <ul>, <li>
        - Ensure proper spacing between sections
      """

      mode_prompts = {
          "NSD": "You will be provided with a calculation problem. Please solve it using the instructions above.",
          "IAQ": ("You are an expert in Indoor Air Quality Engineering (IAQ). You will be provided with a calculation problem. "
                 "Please solve it using appropriate IAQ equations, formulas, specific values, and constants. "
                 "Then, format your answer according to the instructions provided above."),
          "AST": "You are an expert in the Aerosol Science and Technology field. You will be provided with a calculation problem. Please solve it using the instructions above."
      }

      return PROMPT_GSR + mode_prompts.get(mode, mode_prompts[mode])

  def ask_model(self, prompt, mode=None):
      """Generate response from the AI model"""
      try:
        full_prompt = self.prompt_design(mode) + prompt

        completion = self.client.chat.completions.create(
          extra_headers={
              "HTTP-Referer": "Nhan Dinh Ngo",
              "X-Title": "Benchmarking LLM IAQ",
          },
          model=self.model,
          messages=[{"role": "user", "content": full_prompt}],
          max_tokens=self.max_token
        )

        return completion.choices[0].message.content

      except Exception as e:
        print(f"Error in ask_model: {e}")
        return None

  def display_Markdown(self,response):
    display(Markdown(response))

  def save_Markdown(self, response, filename):
          """Save response as Markdown file"""
          try:
              markdown_path = os.path.join(self.output_folder, f"{filename}.md")

              # Add evaluation section
              evaluation_text = """
## YOUR EVALUATION
- [1] Fully understand
- [0.5] Partially understand
- [0] Not Fully understand
- [1] Correct Result + Correct Equation
- [0.75] Wrong Results + Correct Equation
- [0.5] Correct Result + Wrong Equation
- [0] Wrong Results + Wrong Equation
"""

              full_response = response + evaluation_text

              # Save the markdown file
              with open(markdown_path, 'w', encoding='utf-8') as f:
                f.write(full_response)

              print(f"Markdown file saved successfully: {markdown_path}")
              return filename, self.output_folder

          except Exception as e:
              print(f"Error in save_markdown_to_html: {e}")
              return None, None

  def process_problems(self, df, batch_size, answer_mode, max_problems, display, save):
      """Process problems from DataFrame"""
      try:
          for i_batch in range(1, batch_size + 1):
              for index, row in df.iloc[:max_problems].iterrows():
                  example_id = str(row["Example 2"]).strip()
                  problem = str(row["Problems"]).strip()

                  batch_filename = f"Name: {example_id}-{answer_mode}-{self.filename_id}-{i_batch}"
                  print(f"Generating response for {batch_filename}...")

                  # Generate response
                  response = self.ask_model(problem, answer_mode)
                  if not response:
                      print(f"Failed to generate response for {batch_filename}")
                      continue

                  if display == True:
                    # Display result
                    self.display_Markdown(response)

                  if save == True:
                    # Save to Markdown
                    save_filename, save_folder = self.save_Markdown(response, batch_filename)
                    if save_filename:
                        print(f"Successfully saved file {save_filename} to folder {save_folder}")

          print("Done processing all problems")

      except Exception as e:
          print(f"Error in process_problems: {e}")

# Initialize the solver
def initialize_solver():
    """Initialize and return the LLMSolver instance"""
    solver = LLMSolver(
        user_base_url,
        user_api_key,
        user_model,
        user_filename_id,
        user_max_token,
        user_output_folder
    )
    return solver

# Load CSV file
def load_data(solver):
    df = solver.load_data()
    total_rows = len(df)
    print(f"Total rows in dataset: {total_rows}")
    return df, total_rows

# Run model
def run_model(solver, df, batchsize, answermode, maxproblems, displays, saves):
    solver.process_problems(df, batch_size=batchsize, answer_mode=answermode, max_problems=maxproblems, display=displays, save=saves)

In [ ]:
# Initialize the solver
solver = initialize_solver()

# Load CSV data
df, total_problems = load_data(solver)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Markdown files will be saved to: /content/drive/MyDrive/LLM_Nhan/IAQ_DeepSeek_R1_0528_Outputs


Saving 06062025_IAQE_Example_Dataset_CSV.csv to 06062025_IAQE_Example_Dataset_CSV.csv


,Example 1,Example 2,Problems
0,Example 2.1.1,Example 211,An employee works in a building for eight hour...
1,Example 2.1.2,Example 212,An employee works in a building for eight hour...
2,Example 2.1.3,Example 213,An employee works in a building for eight hour...
3,Example 2.2,Example 22,The carbon dioxide concentration in the ground...
4,Example 2.3,Example 23,"In a newly built building, the measured concen..."
5,Example 3.3,Example 33,The total particle count in a building is 100 ...
6,Example 3.4,Example 34,The particle concentration in a room is measur...
7,Example 4.1.1,Example 411,The cross section of the hatch cabinet at a po...
8,Example 4.1.2,Example 412,The cross section of the hatch cabinet at a po...
9,Example 4.1.3,Example 413,The cross section of the hatch cabinet at a po...


Total rows in dataset: 81


In [ ]:
# Run the model
run_model(
    solver,
    df,
    batchsize=5,
    answermode = "IAQ",
    maxproblems = total_problems,
    displays = False,
    saves = True
)

Generating response for Name: Example 211-IAQ-deepseek-r1-1...
Markdown file saved successfully: /content/drive/MyDrive/LLM_Nhan/IAQ_DeepSeek_R1_0528_Outputs/Name: Example 211-IAQ-deepseek-r1-1.md
Successfully saved file Name: Example 211-IAQ-deepseek-r1-1 to folder /content/drive/MyDrive/LLM_Nhan/IAQ_DeepSeek_R1_0528_Outputs
Generating response for Name: Example 212-IAQ-deepseek-r1-1...
Markdown file saved successfully: /content/drive/MyDrive/LLM_Nhan/IAQ_DeepSeek_R1_0528_Outputs/Name: Example 212-IAQ-deepseek-r1-1.md
Successfully saved file Name: Example 212-IAQ-deepseek-r1-1 to folder /content/drive/MyDrive/LLM_Nhan/IAQ_DeepSeek_R1_0528_Outputs
Generating response for Name: Example 213-IAQ-deepseek-r1-1...
Markdown file saved successfully: /content/drive/MyDrive/LLM_Nhan/IAQ_DeepSeek_R1_0528_Outputs/Name: Example 213-IAQ-deepseek-r1-1.md
Successfully saved file Name: Example 213-IAQ-deepseek-r1-1 to folder /content/drive/MyDrive/LLM_Nhan/IAQ_DeepSeek_R1_0528_Outputs
Generating respo